# Frequency & Severity GLMs for Auto Insurance

This notebook builds GLMs for claim **frequency** and **severity**, then combines
them into a pure premium estimate as required in the MA 326 project.  We use
driver, vehicle, territory, and claims-history factors as predictors.


In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import statsmodels.api as sm

# Load pre-cleaned dataset
df = pd.read_csv("motor_cleaned.csv")

print("Loaded cleaned dataset:")
df.head()


Loaded cleaned dataset:


,ID,Date_start_contract,Date_last_renewal,Date_next_renewal,Date_birth,Date_driving_licence,Distribution_channel,Seniority,Policies_in_force,Max_policies,...,Year_matriculation,Power,Cylinder_capacity,Value_vehicle,N_doors,Type_fuel,Length,Weight,Age,Severity
0,1,05/11/2015,05/11/2015,05/11/2016,1956-04-15,20/03/1976,0,4,1,2,...,2004,80,599,7068.0,0,P,NaN,190,68,NaN
1,1,05/11/2015,05/11/2016,05/11/2017,1956-04-15,20/03/1976,0,4,1,2,...,2004,80,599,7068.0,0,P,NaN,190,68,NaN
2,1,05/11/2015,05/11/2017,05/11/2018,1956-04-15,20/03/1976,0,4,2,2,...,2004,80,599,7068.0,0,P,NaN,190,68,NaN
3,1,05/11/2015,05/11/2018,05/11/2019,1956-04-15,20/03/1976,0,4,2,2,...,2004,80,599,7068.0,0,P,NaN,190,68,NaN
4,2,26/09/2017,26/09/2017,26/09/2018,1956-04-15,20/03/1976,0,4,2,2,...,2004,80,599,7068.0,0,P,NaN,190,68,NaN


# Predictor Selection

In [ ]:
predictors = [
    "Age", "Seniority", "Second_driver", "Distribution_channel",
    "Power", "Weight", "Value_vehicle", "Cylinder_capacity",
    "Type_fuel", "Area", "Type_risk",
    "N_claims_history", "R_Claims_history"
]

# Keep rows with no missing predictors
df_model = df[predictors + ["N_claims_year", "Severity"]].dropna()

# One-hot encode categorical vars
cat_cols = ["Type_fuel", "Area", "Type_risk"]
X_full = pd.get_dummies(df_model[predictors], columns=cat_cols, drop_first=True)

# Frequency (all policies)
y_freq = df_model["N_claims_year"]
X_freq = X_full

# Severity target (only for policies with claims)
mask_sev = df_model["Severity"].notna()
y_sev = df_model.loc[mask_sev, "Severity"]
X_sev = X_full.loc[mask_sev]


# Train/Test Split (40/60)

In [42]:
Xf_train, Xf_test, yf_train, yf_test = train_test_split(
    X_freq, y_freq, train_size=0.4, random_state=42
)

Xs_train, Xs_test, ys_train, ys_test = train_test_split(
    X_sev, y_sev, train_size=0.4, random_state=42
)


# Frequency Model

In [43]:
# Add constant
Xf_train_const = sm.add_constant(Xf_train)
Xf_test_const  = sm.add_constant(Xf_test)

print(Xf_train_const.dtypes)
print(yf_train.dtype)

# Convert to plain NumPy float arrays
Xf_train_np = Xf_train_const.to_numpy(dtype=float)
Xf_test_np  = Xf_test_const.to_numpy(dtype=float)
yf_train_np = yf_train.to_numpy(dtype=float)
yf_test_np  = yf_test.to_numpy(dtype=float)

# Fit Poisson GLM
freq_model = sm.GLM(yf_train_np, Xf_train_np, family=sm.families.Poisson())
freq_res   = freq_model.fit()
print(freq_res.summary())

overdispersion = freq_res.deviance / freq_res.df_resid
print("Overdispersion ratio (Poisson):", overdispersion)

# Predictions on test set
freq_pred_test = freq_res.predict(Xf_test_np)
print(freq_pred_test[:10])


const                   float64
Age                       int64
Seniority                 int64
Second_driver             int64
Distribution_channel      int64
Power                     int64
Weight                    int64
Value_vehicle           float64
Cylinder_capacity         int64
N_claims_history          int64
R_Claims_history        float64
Type_fuel_P                bool
Area_1                     bool
Type_risk_2                bool
Type_risk_3                bool
Type_risk_4                bool
dtype: object
int64
                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                 7799
Model:                            GLM   Df Residuals:                     7784
Model Family:                 Poisson   Df Model:                           14
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1

# Severity Model

In [44]:
# Add constant
Xs_train_const = sm.add_constant(Xs_train)
Xs_test_const  = sm.add_constant(Xs_test)

print(Xs_train_const.dtypes)
print(ys_train.dtype)

# Convert to plain NumPy arrays (float)
Xs_train_np = Xs_train_const.to_numpy(dtype=float)
Xs_test_np  = Xs_test_const.to_numpy(dtype=float)
ys_train_np = ys_train.to_numpy(dtype=float)
ys_test_np  = ys_test.to_numpy(dtype=float)

# Gamma needs strictly positive y
mask_sev_pos = ys_train_np > 0
Xs_train_np_pos = Xs_train_np[mask_sev_pos]
ys_train_np_pos = ys_train_np[mask_sev_pos]

# Fit Gamma GLM with log link
sev_model = sm.GLM(
    ys_train_np_pos,
    Xs_train_np_pos,
    family=sm.families.Gamma(link=sm.families.links.log())
)
sev_res = sev_model.fit()
print(sev_res.summary())

sev_pred_test = sev_res.predict(Xs_test_np)
print(sev_pred_test[:10])

const                   float64
Age                       int64
Seniority                 int64
Second_driver             int64
Distribution_channel      int64
Power                     int64
Weight                    int64
Value_vehicle           float64
Cylinder_capacity         int64
N_claims_history          int64
R_Claims_history        float64
Type_fuel_P                bool
Area_1                     bool
Type_risk_2                bool
Type_risk_3                bool
Type_risk_4                bool
dtype: object
float64


c:\Users\madis\anaconda3\Lib\site-packages\statsmodels\genmod\families\links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                 7799
Model:                            GLM   Df Residuals:                     7784
Model Family:                   Gamma   Df Model:                           14
Link Function:                    log   Scale:                          10.505
Method:                          IRLS   Log-Likelihood:                -61487.
Date:                Sun, 23 Nov 2025   Deviance:                       14776.
Time:                        22:59:20   Pearson chi2:                 8.18e+04
No. Iterations:                    21   Pseudo R-squ. (CS):           0.006821
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.1298      0.306     20.034      0.0

# Model Metrics

In [ ]:
# Metrics (use the same test vectors used to train with NumPy)
print("Frequency MSE:", mean_squared_error(yf_test_np, freq_pred_test))
print("Frequency MAE:", mean_absolute_error(yf_test_np, freq_pred_test))

print("Severity MSE:", mean_squared_error(ys_test_np, sev_pred_test))
print("Severity MAE:", mean_absolute_error(ys_test_np, sev_pred_test))

# Pure premium prediction: frequency × severity
pure_premium_pred = freq_pred_test * sev_pred_test

print("Pure premium predictions (first 10):")
print(pure_premium_pred[:10])

# Make as labeled Series
pure_premium_series = pd.Series(pure_premium_pred, name="PurePremium")
print(pure_premium_series.head())

Frequency MSE: 2.2607143993142036
Frequency MAE: 0.9741739811323563
Severity MSE: 2014529.560202747
Severity MAE: 477.94602975029846
Pure premium predictions (first 10):
[1496.75680776  793.69864283  965.21356756  851.30283995  664.91790119
  797.16244499  748.857325    649.0880684   910.78350884 1024.34327838]
0    1496.756808
1     793.698643
2     965.213568
3     851.302840
4     664.917901
Name: PurePremium, dtype: float64
